In [102]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import os
import glob
import codecs
import csv

In [103]:
import glob
import pandas as pd

fds = []
for f in glob.glob("../atp_matches/atp_matches*.csv"):
    fds.append(pd.read_csv(f))

frame = pd.concat(fds)

In [104]:
#frame has all the atp matches
#so made main df - to become csv later
frame

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2.0,NaN,...,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0
1,2019-M020,Brisbane,Hard,32,A,20181231,299,106421,4.0,NaN,...,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,2.0,NaN,...,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0
3,2019-M020,Brisbane,Hard,32,A,20181231,297,104542,NaN,PR,...,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0
4,2019-M020,Brisbane,Hard,32,A,20181231,296,106421,4.0,NaN,...,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2896,2014-605,Tour Finals,Hard,8,F,20141109,514,104925,1.0,NaN,...,38.0,29.0,5.0,11.0,3.0,9.0,1.0,10010.0,5.0,4625.0
2897,2014-605,Tour Finals,Hard,8,F,20141109,515,104925,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10010.0,2.0,8700.0
2898,2014-D015,Davis Cup WG F: FRA vs SUI,Clay,4,D,20141121,1,104527,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5295.0,12.0,2740.0
2899,2014-D015,Davis Cup WG F: FRA vs SUI,Clay,4,D,20141121,2,104792,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19.0,1825.0,2.0,9700.0


In [105]:
### BUILT ATPTotal for all ATP matches combined
#frame.to_csv('ATPTotal.csv')

In [106]:
# save under new name
ATP = frame

In [107]:
# Win Count DF
win_count = ATP.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
win_count = win_count.rename(columns={'winner_name': 'win_count', 'winner_id' : 'PlayerID'})
win_count

,PlayerID,win_count
0,100002,3
1,100003,10
2,100004,4
3,100005,126
4,100007,4
...,...,...
3855,209870,1
3856,209874,1
3857,210013,158
3858,210079,1


In [108]:
# Loss Count DF
loss_count = ATP.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
loss_count = loss_count.rename(columns={'loser_name': 'lose_count', 'loser_id' : 'PlayerID'})
loss_count

,PlayerID,lose_count
0,100001,2
1,100002,5
2,100003,17
3,100004,10
4,100005,54
...,...,...
6349,209903,1
6350,209916,1
6351,210013,212
6352,210250,1


In [112]:
#First Serve win percentage added new column
ATP['First_Serve'] = ATP['w_1stIn'] / ATP['w_svpt']
#First Serve Win
ATP['First_Serve_Win'] = ATP['w_1stWon'] / ATP['w_1stIn']
#Break Points? (have mason define)

#### questions:
- Breakpoints and other feature engineering
- Should we avoid point system (cause overfitting?)
- Trim DF at what amount of game wins?
- 

In [110]:
ATP.head(1)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,FirstServe,First_Serve_Win
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2.0,NaN,...,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,0.571429,0.704545


In [ ]:
### built this out in feature building notebook

In [120]:
#Create dataframe for Player Data and label columns
PlayerColumns = ['PlayerID', 'Player_Name', 'Age', 'Height', 'MaxRank', 'Hand', 'Country', 'First_Serve_%', 'First_Serve_Win_%']

PlayerData = pd.DataFrame(columns = PlayerColumns)
PlayerData.set_index('PlayerID')


#Add data to the PlayerData dataframe
for index, row in ATP.iterrows():
    #player was the winner
    p_id = int(row.winner_id)
    
    #Check value
    if(p_id not in PlayerData.index):
        PlayerData.loc[p_id] = [p_id, row.winner_name, row.winner_age, row.winner_ht, row.winner_rank, row.winner_hand, row.winner_ioc, row.First_Serve, row.First_Serve_Win]

    #Update rank with best rank value
    if(row.winner_rank < PlayerData.loc[p_id].MaxRank and not np.isnan(PlayerData.loc[p_id].MaxRank)):
        PlayerData.loc[p_id] = [p_id, row.winner_name, row.winner_age, row.winner_ht, row.winner_rank, row.winner_hand, row.winner_ioc, row.First_Serve, row.First_Serve_Win]

    #Player was loser
    p2_id = int(row.loser_id)
    #Check for loser
    if(p2_id not in PlayerData.index):
        PlayerData.loc[p2_id] = [p2_id, row.loser_name, row.loser_age, row.loser_ht, row.loser_rank, row.loser_hand, row.loser_ioc, row.First_Serve, row.First_Serve_Win]

    #Update rank with best rank value
    if(row.loser_rank < PlayerData.loc[p2_id].MaxRank and not np.isnan(PlayerData.loc[p2_id].MaxRank)):
        PlayerData.loc[p2_id] = [p2_id, row.loser_name, row.loser_age, row.loser_ht, row.loser_rank, row.loser_hand, row.loser_ioc, row.First_Serve, row.First_Serve_Win]


In [121]:
PlayerData

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,First_Serve_%,First_Serve_Win_%
105453,105453,Kei Nishikori,26.839151,178.0,4.0,R,JPN,0.692308,0.722222
106421,106421,Daniil Medvedev,25.434634,198.0,2.0,R,RUS,NaN,NaN
104542,104542,Jo-Wilfried Tsonga,26.863792,188.0,5.0,R,FRA,0.712329,0.730769
104871,104871,Jeremy Chardy,25.998631,188.0,25.0,R,FRA,0.570093,0.704918
200282,200282,Alex De Minaur,22.360027,183.0,15.0,R,AUS,0.622378,0.764045
...,...,...,...,...,...,...,...,...,...
105059,105059,Kento Takeuchi,26.759754,NaN,441.0,L,JPN,0.523810,0.772727
104393,104393,Gregoire Burquier,30.206708,NaN,205.0,L,FRA,0.625000,0.666667
105999,105999,Bowen Ouyang,22.343600,NaN,1098.0,L,CHN,0.700000,0.732143
106321,106321,Xin Gao,20.364134,NaN,944.0,U,CHN,0.566667,0.725490


In [122]:
### money for merge

from functools import reduce

data_frames = [PlayerData, win_count, loss_count]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
df_merged

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,First_Serve_%,First_Serve_Win_%,win_count,lose_count
0,105453,Kei Nishikori,26.839151,178.0,4.0,R,JPN,0.692308,0.722222,434.0,217.0
1,106421,Daniil Medvedev,25.434634,198.0,2.0,R,RUS,NaN,NaN,222.0,98.0
2,104542,Jo-Wilfried Tsonga,26.863792,188.0,5.0,R,FRA,0.712329,0.730769,471.0,234.0
3,104871,Jeremy Chardy,25.998631,188.0,25.0,R,FRA,0.570093,0.704918,299.0,303.0
4,200282,Alex De Minaur,22.360027,183.0,15.0,R,AUS,0.622378,0.764045,111.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...
6485,105059,Kento Takeuchi,26.759754,NaN,441.0,L,JPN,0.523810,0.772727,NaN,1.0
6486,104393,Gregoire Burquier,30.206708,NaN,205.0,L,FRA,0.625000,0.666667,NaN,1.0
6487,105999,Bowen Ouyang,22.343600,NaN,1098.0,L,CHN,0.700000,0.732143,NaN,1.0
6488,106321,Xin Gao,20.364134,NaN,944.0,U,CHN,0.566667,0.725490,NaN,1.0


In [130]:
# adding Win%
df_merged['Win%'] = round((df_merged['win_count']/ df_merged['lose_count']),2)

In [131]:
### SAVE IT FOR CSV (MANY UPDATES NEEDED)
df_merged.to_csv('PlayerData.csv')

In [129]:
df_merged[df_merged['MaxRank']==1]

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,First_Serve_%,First_Serve_Win_%,win_count,lose_count,Win%
10,104918,Andy Murray,29.503080,190.0,1.0,R,GBR,0.534483,0.741935,700.0,219.0,3.20
30,104925,Novak Djokovic,31.611225,188.0,1.0,R,SRB,0.750000,0.787879,1002.0,199.0,5.04
109,104745,Rafael Nadal,33.440110,185.0,1.0,L,ESP,0.644860,0.724638,1036.0,214.0,4.84
110,103819,Roger Federer,36.572211,185.0,1.0,R,SUI,0.646341,0.735849,1259.0,280.0,4.50
583,102845,Carlos Moya,22.546201,190.0,1.0,R,ESP,0.356322,0.548387,578.0,322.0,1.80
600,104053,Andy Roddick,21.349760,188.0,1.0,R,USA,0.660714,0.810811,612.0,215.0,2.85
609,103498,Marat Safin,21.065024,193.0,1.0,R,RUS,0.580645,0.851852,424.0,269.0,1.58
629,103507,Juan Carlos Ferrero,23.665982,183.0,1.0,R,ESP,NaN,NaN,479.0,262.0,1.83
669,103720,Lleyton Hewitt,21.289528,180.0,1.0,R,AUS,0.552239,0.702703,619.0,264.0,2.34
1162,100581,John McEnroe,22.480493,180.0,1.0,L,USA,NaN,NaN,886.0,201.0,4.41


Player ID use as a foreign winner id and loser id 

pd.merge pass an array of column names - to join across multiple columns -